In [3]:
!apt-get update
!apt-get install -y openjdk-11-jdk
!pip install pyspark==3.5.1 kafka-python
!wget https://archive.apache.org/dist/kafka/3.6.1/kafka_2.13-3.6.1.tgz
!tar -xzf kafka_2.13-3.6.1.tgz
!kafka_2.13-3.6.1/bin/zookeeper-server-start.sh -daemon kafka_2.13-3.6.1/config/zookeeper.properties
!kafka_2.13-3.6.1/bin/kafka-server-start.sh -daemon kafka_2.13-3.6.1/config/server.properties
!kafka_2.13-3.6.1/bin/kafka-topics.sh \
--create \
--topic suicide-topic \
--bootstrap-server localhost:9092 \
--partitions 1 \
--replication-factor 1

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://cli.github.com/packages stable/main amd64 Packages [354 B]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,870 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,623 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 Packages [38.5 kB]
Get:14 http

In [4]:
from kafka import KafkaProducer
import json

producer = KafkaProducer(
    bootstrap_servers="localhost:9092",
    value_serializer=lambda v: json.dumps(v).encode("utf-8")
)

producer.send("suicide-topic", {"text": "test kafka"})
producer.flush()




In [5]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .master("local[*]")
    .appName("BigDataKafka")
    .config(
        "spark.jars.packages",
        "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1"
    )
    .getOrCreate()
)

spark.sparkContext.setLogLevel("WARN")


In [6]:
kafka_df = (
    spark.readStream
    .format("kafka")
    .option("kafka.bootstrap.servers", "localhost:9092")
    .option("subscribe", "suicide-topic")
    .load()
)




In [7]:
query = (
    kafka_df
    .selectExpr("CAST(value AS STRING)")
    .writeStream
    .format("console")
    .outputMode("append")
    .start()
)

# query.awaitTermination()


In [8]:
from pyspark.sql.types import StructType, StructField, StringType

schema = StructType([
    StructField("text", StringType(), True)
])


In [9]:
from pyspark.sql.functions import from_json, col

kafka_parsed = kafka_df.select(
    from_json(col("value").cast("string"), schema).alias("data")
).select("data.*")


In [10]:

from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("ProjektBigDataKafka")
    .master("local[*]")
    .config(
        "spark.jars.packages",
        "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1"
    )
    .getOrCreate()
)

spark.sparkContext.setLogLevel("WARN")


In [11]:
!pip install streamlit


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 35.2 MB/s eta 0:00:00


In [12]:
!unzip /content/suicidewatch_linearSVC_spark_pipeline.zip >

/bin/bash: -c: line 1: syntax error near unexpected token `newline'
/bin/bash: -c: line 1: `unzip /content/suicidewatch_linearSVC_spark_pipeline.zip >'


In [13]:
%%writefile ui.py
import streamlit as st
from pyspark.sql import SparkSession
from pyspark.ml import PipelineModel

# --- CONFIG (MUSI BYĆ NA GÓRZE) ---
st.set_page_config(
    page_title="Suicide Ideation Detection",
    page_icon="🧠",
    layout="centered"
)

st.markdown("""
<style>
body {
    background-color: #f5f7fa;
}
.stButton>button {
    background-color: #4a6fa5;
    color: white;
    border-radius: 8px;
}
</style>
""", unsafe_allow_html=True)
# ---------------------------------

spark = SparkSession.builder.getOrCreate()
model = PipelineModel.load("content/sample_data")

st.title("Suicide Ideation Detection")

st.markdown("""
### About the project
This application detects **suicidal ideation** in user-generated text
using a **Linear SVM model** trained on Reddit data.
""")

st.divider()

text = st.text_area("Enter text:")

if st.button("Predict"):
    if text.strip():
        df = spark.createDataFrame([(text,)], ["text_raw"])
        pred = model.transform(df).select("prediction").collect()[0][0]

        if pred == 1.0:
            st.error("⚠️ Prediction: SUICIDAL")
        else:
            st.success("✅ Prediction: NON-SUICIDAL")
    else:
        st.warning("Please enter some text.")




Overwriting ui.py


In [14]:
!pkill -f streamlit
!pkill -f cloudflared


In [19]:
!nohup streamlit run ui.py \
  --server.port 8501 \
  --server.address 0.0.0.0 \
  --server.enableCORS false \
  --server.enableXsrfProtection false \
  > streamlit.log 2>&1 &



In [16]:
!ps aux | grep streamlit


root        3826 10.0  0.1  27920 19076 ?        R    17:07   0:00 /usr/bin/python3 /usr/local/bin/streamlit run ui.py --server.port 8501 --server.address 0.0.0.0 --server.enableCORS false --server.enableXsrfProtection false
root        3828  0.0  0.0   7372  3444 ?        S    17:07   0:00 /bin/bash -c ps aux | grep streamlit
root        3830  0.0  0.0   6480  2436 ?        S    17:07   0:00 grep streamlit


In [21]:
!nohup ./cloudflared-linux-amd64 tunnel --url http://localhost:8501 \
  > cloudflare.log 2>&1 &



In [18]:
!grep trycloudflare cloudflare.log


In [22]:
!pkill -f streamlit
!nohup streamlit run ui.py --server.port 8501 --server.address 0.0.0.0 > streamlit.log 2>&1 &
!tail -n 20 streamlit.log


In [24]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
!ls -lh cloudflared-linux-amd64


--2026-01-18 17:18:31--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.11.1/cloudflared-linux-amd64 [following]
--2026-01-18 17:18:31--  https://github.com/cloudflare/cloudflared/releases/download/2025.11.1/cloudflared-linux-amd64
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/106867604/955e9d1b-ac5e-4188-8867-e5f53958a8fe?sp=r&sv=2018-11-09&sr=b&spr=https&se=2026-01-18T18%3A10%3A59Z&rscd=attachment%3B+filename%3Dcloudflared-linux-amd64&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2026-01-18

In [27]:
!pkill -f cloudflared
!nohup ./cloudflared-linux-amd64 tunnel --url http://localhost:8501 \
  > cloudflare.log 2>&1 &
!sleep 3
!tail -n 50 cloudflare.log


2026-01-18T17:18:55Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2026-01-18T17:18:55Z INF Requesting new quick Tunnel on trycloudflare.com...


In [28]:
!grep -o 'https://[-a-z0-9]*\.trycloudflare\.com' cloudflare.log | head -n 1


https://optics-williams-established-retrieval.trycloudflare.com
